<a href="https://colab.research.google.com/github/sonii26/API/blob/main/Implement_Q%26A_Model_with_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Basic Keyword Matching QA (No ML)

In [1]:
context = """
PseudoClan is a next-gen IT company transforming business ideas into intelligent digital solutions.
We’re a clan of passionate engineers, developers, and creators who believe in solving real-world challenges
with tech that’s scalable, secure, and smart.
"""

question = "How does PseudoClan transform business ideas into intelligent, scalable, and secure digital solutions?"
keywords = ["transform", "business", "intelligent", "digital", "solutions"]

# Split and search
sentences = context.split(".")
best_sentence = ""
max_matches = 0

for sentence in sentences:
    match_count = sum(1 for word in keywords if word.lower() in sentence.lower())
    if match_count > max_matches:
        max_matches = match_count
        best_sentence = sentence.strip()

# Display answer
if best_sentence:
    print("Answer:", best_sentence)
else:
    print("No relevant answer found.")


Answer: PseudoClan is a next-gen IT company transforming business ideas into intelligent digital solutions


 Using HuggingFace Q&A Pipeline

In [2]:
from transformers import pipeline

qa = pipeline("question-answering")

context = """
PseudoClan is a next-gen IT company transforming business ideas into intelligent digital solutions.
We’re a clan of passionate engineers, developers, and creators who believe in solving real-world challenges
with tech that’s scalable, secure, and smart.
"""

question = "What does PseudoClan do?"

result = qa({
    "context": context,
    "question": question
})

print("Answer:", result['answer'])


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Answer: transforming business ideas into intelligent digital solutions


 Specific Model with Q&A

In [3]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad")

context = """
PseudoClan is a next-gen IT company transforming business ideas into intelligent digital solutions.
We’re a clan of passionate engineers, developers, and creators who believe in solving real-world challenges
with tech that’s scalable, secure, and smart.
"""

question = "What does PseudoClan do?"

result = qa_pipeline({
    "context": context,
    "question": question
})

print("Answer:", result["answer"])


Device set to use cpu


Answer: transforming business ideas into intelligent digital solutions


 FastAPI Deployment with ngrok

In [57]:
from pyngrok import ngrok
ngrok.kill()

In [58]:
# Install dependencies (if needed)
!pip install -q fastapi uvicorn transformers torch pyngrok


In [59]:
from pyngrok import ngrok
ngrok.set_auth_token("2yShxu6XhJhYAiXxl1XO6VVPKgL_72tfC1wDAqGhDDWj7J8PY")


In [60]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline
import uvicorn
import threading

qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

app = FastAPI()

class QARequest(BaseModel):
    context: str
    question: str

@app.get("/")
def read_root():
    return {"message": "Welcome to the Q&A API running in Google Colab!"}

@app.post("/answer")
def answer_question(data: QARequest):
    result = qa_pipeline({
        "context": data.context,
        "question": data.question
    })
    return {"answer": result['answer']}
public_url = ngrok.connect(8000)
print(f"🚀 Public API URL: {public_url}/docs")
def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Run in a separate thread
threading.Thread(target=start_server).start()


Device set to use cpu


🚀 Public API URL: NgrokTunnel: "https://e2ff-34-73-186-64.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     Started server process [1826]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


 Send Request to Test the API

In [64]:
import time
import requests

# Wait for server to start
time.sleep(5)

# Step 6: Send test request to API
payload = {
    "context": "PseudoClan is a next-gen IT company transforming business ideas into intelligent digital solutions.",
    "question": "What does PseudoClan do?"
}

response = requests.post(f"{public_url}/answer", json=payload)

# Print the output
print("\n🧠 Answer:", response.json())


INFO:     34.73.186.64:0 - "POST /answer HTTP/1.1" 200 OK

🧠 Answer: {'answer': 'transforming business ideas into intelligent digital solutions'}


In [62]:
# Start ngrok tunnel
tunnel = ngrok.connect(8000)
public_url = tunnel.public_url  # ✅ this gives just the URL string

print(f"🚀 Public API URL: {public_url}/docs")


🚀 Public API URL: https://1130-34-73-186-64.ngrok-free.app/docs


In [63]:
response = requests.post(f"{public_url}/answer", json=payload)


INFO:     34.73.186.64:0 - "POST /answer HTTP/1.1" 200 OK


In [65]:
{
  "context": "PseudoClan is a next-gen IT company transforming business ideas into intelligent digital solutions.",
  "question": "What does PseudoClan do?"
}

{'context': 'PseudoClan is a next-gen IT company transforming business ideas into intelligent digital solutions.',
 'question': 'What does PseudoClan do?'}